In [5]:
import yt_dlp
import concurrent.futures

In [6]:
def download_youtube_video(url, save_path="./", cookies_path="cookies.txt"):
    try:
        ydl_opts = {
            'outtmpl': f"{save_path}/%(title)s.%(ext)s",
            'format': 'bestvideo+bestaudio/best',
            # 'cookies': cookies_path  # Provide the cookies file
            'cookiesfrombrowser': ('chrome',)
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print("Download completed!")
    except Exception as e:
        print(f"An error occurred: {e}")

In [7]:
def get_latest_videos_from_channel(channel_url, playlist_items = '1-2'):
    ydl_opts = {
        'quiet': False,  # 显示调试信息，帮助我们理解问题
        'extract_flat': True,  # 只提取视频信息，不下载视频
        'playlist_items': playlist_items, 
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            # 提取频道信息
            info_dict = ydl.extract_info(channel_url, download=False)

            # 获取视频条目列表
            videos = info_dict.get('entries', [])

            if not videos:
                print("No videos found.")
                return []

            # 提取所有视频链接
            video_links = [video['url'] for video in videos]

            return video_links

        except Exception as e:
            print(f"Error occurred: {e}")
            return []

In [8]:
def download_latest_videos_concurrently(channel_url, playlist_items='1-2'):
    """获取并并行下载频道的最新视频"""
    video_urls = get_latest_videos_from_channel(channel_url, playlist_items)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(download_youtube_video, video_url) for video_url in video_urls]
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"An error occurred during download: {e}")

In [9]:
channel_url = 'https://www.youtube.com/@NBA/videos'
download_latest_videos_concurrently(channel_url, playlist_items='1-10')

[youtube:tab] Extracting URL: https://www.youtube.com/@NBA/videos
[youtube:tab] @NBA/videos: Downloading webpage
[download] Downloading playlist: NBA - Videos
[youtube:tab] Playlist NBA - Videos: Downloading 10 items
[download] Downloading item 1 of 10
[download] Downloading item 2 of 10
[download] Downloading item 3 of 10
[download] Downloading item 4 of 10
[download] Downloading item 5 of 10
[download] Downloading item 6 of 10
[download] Downloading item 7 of 10
[download] Downloading item 8 of 10
[download] Downloading item 9 of 10
[download] Downloading item 10 of 10
[download] Finished downloading playlist: NBA - Videos
Extracting cookies from chrome
Extracted 3079 cookies from chrome
Extracting cookies from chrome
[youtube] Extracting URL: https://www.youtube.com/watch?v=d_ymmP2S0ck
[youtube] d_ymmP2S0ck: Downloading webpage
[youtube] d_ymmP2S0ck: Downloading ios player API JSON
[youtube] d_ymmP2S0ck: Downloading mweb player API JSON
[youtube] d_ymmP2S0ck: Downloading m3u8 inform